### [Premade Estimators](https://www.tensorflow.org/beta/tutorials/estimators/premade_estimators)

In [1]:
from __future__ import print_function, absolute_import, division, unicode_literals

import tensorflow as tf
import pandas as pd
import numpy as np

In [2]:
import logging
logger = tf.get_logger()
logger.setLevel(logging.ERROR)

#### Dataset

In [3]:
COLUMN_NAMES = ['SepalLength', 'SepalWidth', 'PetalLength', 'PetalWidth', 'Species']
SPECIES = ['Setosa', 'Versicolor', 'Virginica']

In [4]:
train_path = tf.keras.utils.get_file('iris_training.csv', 'https://storage.googleapis.com/download.tensorflow.org/data/iris_training.csv')
test_path = tf.keras.utils.get_file('iris_test.csv', 'https://storage.googleapis.com/download.tensorflow.org/data/iris_test.csv')

In [5]:
train = pd.read_csv(train_path, names=COLUMN_NAMES, header=0)
test = pd.read_csv(test_path, names=COLUMN_NAMES, header=0)

In [6]:
train.head()

,SepalLength,SepalWidth,PetalLength,PetalWidth,Species
0,6.4,2.8,5.6,2.2,2
1,5.0,2.3,3.3,1.0,1
2,4.9,2.5,4.5,1.7,2
3,4.9,3.1,1.5,0.1,0
4,5.7,3.8,1.7,0.3,0


In [7]:
y_train = train.pop('Species')
y_test = test.pop('Species')

#### Create input functions

In [8]:
def input_evaluation_set():
    features = {'SepalLength': np.array([6.4, 5.0]),
               'SepalWidth': np.array([2.8, 2.3]),
               'PetalLength': np.array([5.6, 3.3]),
               'PetalWidth': np.array([2.2, 1.0])}
    labels = np.array([2, 1])
    return features, labels

In [9]:
def input_fn(features, labels, training=True, batch_size=256):
    dataset = tf.data.Dataset.from_tensor_slices((dict(features), labels))
    if training:
        dataset = dataset.shuffle(1000).repeat()
    return dataset.batch(batch_size)

#### Feature columns

In [10]:
feature_cols = []
for key in train.keys():
    feature_cols.append(tf.feature_column.numeric_column(key=key))

#### Instantiate an estimator

In [11]:
classifier = tf.estimator.DNNClassifier(feature_columns=feature_cols,
                                       hidden_units=[10, 10],
                                       n_classes=3)

#### Train the model

In [12]:
classifier.train(input_fn=lambda: input_fn(train, y_train, training=True), steps=5000)

#### Evaluation

In [13]:
eval_result = classifier.evaluate(input_fn=lambda: input_fn(test, y_test, training=False))

In [14]:
print('Test set accuracy: {:.3f}'.format(eval_result['accuracy']))

Test set accuracy: 0.700


#### Making predictions from the trained model

In [15]:
expected = ['Setosa', 'Versicolor', 'Virginica']
predict_x = {
    'SepalLength': [5.1, 5.9, 6.9],
    'SepalWidth': [3.3, 3.0, 3.1],
    'PetalLength': [1.7, 4.2, 5.4],
    'PetalWidth': [0.5, 1.5, 2.1],
}

In [16]:
def input_fn(features, batch_size=256):
    return tf.data.Dataset.from_tensor_slices(dict(features)).batch(batch_size)

In [17]:
predictions = classifier.predict(input_fn=lambda: input_fn(predict_x))

In [18]:
for pred_dict, expec in zip(predictions, expected):
    class_id = pred_dict['class_ids'][0]
    probability = pred_dict['probabilities'][class_id]

    print('Prediction is {} ({:.2f}%), expected {}'.format(SPECIES[class_id], 100*probability, expec))

Prediction is Setosa (67.93%), expected Setosa
Prediction is Virginica (41.70%), expected Versicolor
Prediction is Virginica (60.08%), expected Virginica
